In [1]:
import pandas as pd
from gensim.models import Word2Vec

In [2]:
w2v_data = pd.read_csv('/Users/charuyskaya/domain_timestamp.csv', '\t')
w2v_data.domain_timestamp = w2v_data.domain_timestamp.apply(eval)

In [18]:
import pandas as pd
import os
from tqdm import tqdm
# from nltk.corpus import stopwords
import nltk.data
import logging
import numpy as np  # Make sure that numpy is imported
from gensim.models.word2vec import Word2Vec 
from sklearn.ensemble import RandomForestClassifier

# from KaggleWord2VecUtility import KaggleWord2VecUtility

,Unnamed: 0,domain_timestamp,uid,gender,age


In [19]:
model = Word2Vec(w2v_data.domain_timestamp.values.tolist(),
                 workers=8,
                 size=300,
                 window=10,
                 sample=1e-3)

In [21]:
import numpy as np

def text_to_vec(words, model, size):
    text_vec = np.zeros((size,), dtype="float32")
    n_words = 0

    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model[word])
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec

def texts_to_vecs(texts, model, size):
    texts_vecs = np.zeros((len(texts), size), dtype="float32")
    
    for i, text in enumerate(texts):
        texts_vecs[i] = text_to_vec(text, model, size)

    return texts_vecs

In [59]:
sum_100 = w2v_data.domain_timestamp[:100].sum()

In [ ]:
train_vecs = texts_to_vecs(sum_100, model, 300)

In [33]:
w2v_data['gender'][:10]

0    F
1    M
2    F
3    F
4    M
5    F
6    F
7    F
8    F
9    M
Name: gender, dtype: object

In [53]:
test_df = w2v_data.drop(['gender', 'age', 'Unnamed: 0'], axis=1)

In [58]:
w2v_data[100:110]

,Unnamed: 0,domain_timestamp,uid,gender,age
100,29675,"[yaplakal.com, 1413515245000, yaplakal.com, 14...",03537152-7a6a-435b-b9cc-c502e27e8517,M,35-44
101,30527,"[big-cards.a5ltd.com, 1413520156000, vk.com, 1...",0355d721-ad22-4730-b8de-27cf43795b28,F,25-34
102,30860,"[news.zborg.ru, 1413520944000, govoru.com, 141...",03569aea-0e76-4f66-83ed-43a80b25beed,M,18-24
103,31157,"[bw4u.ru, 1413995620000, powerrangers.ru, 1414...",0357c4f3-e700-4926-a616-c137b8768b96,F,25-34
104,31165,"[znanija.com, 1413563554000, znanija.com, 1413...",03583ff2-7c79-4e86-9084-f4c88963a84b,M,35-44
105,31421,"[b.kavanga.ru, 1426763694000, qxfp15q8ajdisl.r...",0359a781-f574-4798-9462-62fa08bdee3c,M,25-34
106,31423,"[blog-fiesta.com, 1415208445000, blog-fiesta.c...",035e7c3b-26c1-424b-b75c-566192c07fc5,F,35-44
107,31507,"[bigpicture.ru, 1413483797000, bigpicture.ru, ...",035f359d-223c-4be2-9b4a-a44b14d4157c,M,25-34
108,32757,"[mail.rambler.ru, 1426790756000, mail.rambler....",035f742c-8135-49d1-b0b3-1260858b0538,M,25-34
109,32965,"[ololo.fm, 1415823450000, ololo.fm, 1415823451...",03634065-874e-4110-8f9a-a566f689b659,F,35-44


In [54]:
test = texts_to_vecs(test_df.domain_timestamp[100:110].values.tolist(), model, 300)

In [50]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100, n_jobs=8)
forest = forest.fit(train_vecs, w2v_data.gender[:3730])

ValueError: setting an array element with a sequence.

In [55]:
predict = forest.predict(test)

In [56]:
predict

array(['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'], dtype=object)